In [1]:
import torch
import torchvision.transforms as transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import Image, ImageDraw
import os
import numpy as np
import cv2

In [91]:
# 데이터셋 경로 설정
dataset_path = r'D:\yolo\crack-2'

In [92]:
# 이미지 변환
transform = transforms.Compose([transforms.ToTensor()])

In [93]:
# 테스트 데이터셋 로딩
test_image_folder = os.path.join(dataset_path, 'test', 'images2')
test_label_folder = os.path.join(dataset_path, 'test', 'labels')
test_dataset = ImageFolder(root = test_image_folder, transform = transform, target_transform = None)

In [94]:
# DataLoader 설정
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [95]:
# Faster R-CNN 모델 로드
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [96]:
# 결과 시각화 함수 정의
def draw_boxes(image, boxes):
    draw = ImageDraw.Draw(image)
    for box in boxes:
        draw.rectangle(xy=[(box[0], box[1]), (box[2], box[3])], outline="red")

In [129]:
# 테스트 이미지 선택 및 추론
for test_image, _ in test_loader:  # DataLoader에서 랜덤 이미지 선택
    with torch.no_grad():
        predictions = model(test_image)

    # 결과 시각화
    for i in range(len(predictions)):
        image = transforms.ToPILImage()(test_image[i])
        boxes = predictions[i]['boxes'].cpu().numpy()
        draw_boxes(image, boxes)
        image.show()  # 이미지 출력
    break  # 첫 번째 이미지만 선택하고 반복문 종료